# Assembly of system with multiple domains, variables and numerics

This tutorial has the dual purpose of illustrating parameter assigment in PorePy, and also showing  how to set up problems in (mixed-dimensional) geometries. It contains two examples, one covering a simple setup (the pressure equation), and a second illustrating the full generality of the coupling scheme. 


In [1]:
import numpy as np
import scipy.sparse as sps

import porepy as pp

## Data assignment
We will mainly use default values for parameters, while overriding some of the values. Sets of default parameters are available for flow, transport and mechanics (elasticity). For example, initialize_default_data initializes a second order permeability tensor in the flow data, and a fourth order stiffness tensor in the mechanics data. For more details, and definitions of what the defaults are, see the modules pp/params/data.py and pp/params/parameter_dictionaries.py <br>
 The parameters are stored in a class pp.Parameters. This class is again stored in the data dictionary on each node and edge in the GridBucket, that is, in the variable d in this loop. The Paramater object can be accessed by `d[pp.PARAMETERS]`. To allow storage of parameters for several problems simultaneously (say, we want to solve a combined flow and transport problem), the Parameter class uses keywords to identify sets of parameters. This keyword must also be provided to the discretization method.
When the parameter class is initialized with default values, the default behavior is to identify the parameters by the same keyword as is used to choose the type of default parameters (`default_parameter_type`). While this is usually good practice, we here override this behavior for illustrative purposes, using the `keyword_param_storage`.

In [2]:
def assign_data(gb, keyword_param_storage):
    # Method to assign data.
    for g, d in gb:
        # This keyword is used to define which set of default parameters to pick
        # Replace with 'transport' or 'mechanics' if needed
        default_parameter_type = 'flow'  

        # Assign a non-default permeability, for illustrative purposes
        if g.dim == 2:
            kxx = 10 * np.ones(g.num_cells)
        else:
            kxx = 0.1 * np.ones(g.num_cells)

        perm = pp.SecondOrderTensor(kxx)
        
        # We also set Dirichlet conditions, as the default Neumann condition
        # gives a singular problem
        bc = pp.BoundaryCondition(g, g.get_boundary_faces(), 'dir')

        # Create a dictionary to override the default parameters
        specified_parameters = {'second_order_tensor': perm, 'bc': bc}

        # Define the 
        pp.initialize_default_data(g, d, default_parameter_type, specified_parameters,
                                   keyword_param_storage)

        # Internally to the Parameter class, the parameters are stored as dictionaries.
        # To illustrate how to access specific sets of parameters, print the keywords
        # for one of the grids
        if g.dim == 2:
            print('The assigned parameters for the 2d grid are')
            print(d[pp.PARAMETERS][keyword_param_storage].keys())
           
    for e, d in gb.edges():
        # On edges in the GridBucket, there is currently no methods for default initialization.

        data = {"normal_diffusivity": 2e1}
        # Add parameters: We again use keywords to identify sets of parameters.
        d[pp.PARAMETERS] = pp.Parameters(keywords=['flow_param_edge'], dictionaries=[data])
    
    return gb

# Example 1
The practical way of setting up a problem with a single variable is described here. For explanations, and hints on how to consider a more general setting, see the expanded Example 2 below. <br>
As shown in the tutorial on single-phase flow, the equation in the mono-dimensional case is
$$ - \nabla \cdot K \nabla p = f. $$
We expand to the mixed-dimensional version of the single-phase flow problem by solving the problem in each of the subdomains (here: fracture and matrix) and adding the flux between the subdomains
$$ \lambda = - \kappa (p_{fracture} - \texttt{tr }p_{matrix}), $$ 
with $\kappa$ denoting the normal permeability of the fractures. For details, refer to the tutorial on single-phase flow and published papers, e.g. [this one](https://arxiv.org/abs/1802.05961).<br><br>
We start by defining the grid bucket and assigning parameters, tagging them with a keyword. This keyword ensures that the discretizer (here tpfa, defined below) uses the right set of parameters. 

In [3]:
gb, _ = pp.grid_buckets_2d.single_horizontal([2, 2], simplex=False)

parameter_keyword = 'flow_param'
gb = assign_data(gb, parameter_keyword)

The assigned parameters for the 2d grid are
dict_keys(['source', 'mass_weight', 'second_order_tensor', 'bc', 'bc_values', 'time_step'])


Now, we define the variables on grids and edges and identify the individual terms of the equation we want to solve. We have an equation for the pressure on each grid (node of the GridBucket), and an equation for the mortar flux between them (edge of the bucket). The terms to be discretized are the diffusion term on the nodes ($- \nabla \cdot K \nabla p$) and the coupling term $- \kappa (p_{fracture} - \texttt{tr }p_{matrix})$ on the edges.

In [4]:
# Define the pressure variable with the same keyword on all grids
grid_variable = 'pressure'
# Variable name for the flux between grids, that is, the primary variable
# on the edges in the GridBucket.
mortar_variable = 'mortar_flux'

# Identifier of the discretization operator on each grid
operator_keyword = 'diffusion'
# Identifier of the discretization operator between grids
coupling_operator_keyword = 'coupling_operator'

# Use a two-point flux approximation on all grids.
# Note the keyword here: It must be the same as used when assigning the 
# parameters.
tpfa = pp.Tpfa(parameter_keyword)

# Between the grids we use a Robin type coupling (resistance to flow over a fracture).
# Again, the keyword must be the same as used to assign data to the edge
# The edge discretization also needs access to the corresponding discretizations 
# on the neighboring nodes
edge_discretization = pp.RobinCoupling('flow_param_edge', tpfa, tpfa)

# Loop over the nodes in the GridBucket, define primary variables and discretization schemes
for g, d in gb:
    # Assign primary variables on this grid. It has one degree of freedom per cell.
    d[pp.PRIMARY_VARIABLES] = {grid_variable: {"cells": 1, "faces": 0}}
    # Assign discretization operator for the variable.
    # If the discretization is composed of several terms, they can be assigned
    # by multiple entries in the inner dictionary, e.g.
    #  {operator_keyword_1: method_1, operator_keyword_2: method_2, ...}
    d[pp.DISCRETIZATION] = {grid_variable: {operator_keyword: tpfa}}
    
# Loop over the edges in the GridBucket, define primary variables and discretizations
for e, d in gb.edges():
    g1, g2 = gb.nodes_of_edge(e)
    # The mortar variable has one degree of freedom per cell in the mortar grid
    d[pp.PRIMARY_VARIABLES] = {mortar_variable: {"cells": 1}}
    
    # The coupling discretization links an edge discretization with variables
    # and discretization operators on each neighboring grid
    d[pp.COUPLING_DISCRETIZATION] = {
        coupling_operator_keyword: {
            g1: (grid_variable, operator_keyword),
            g2: (grid_variable, operator_keyword),
            e: (mortar_variable, edge_discretization),
        }
    }
    d[pp.DISCRETIZATION_MATRICES] = {'flow_param_edge': {}}

The task of assembling the linear system is left to a dedicated object, called an `Assembler`, whih again relies on a `DofManager` to keep track of the ordering of unknowns (for more, see below).

Discretization and assembly of the global linear system can in this case be carried out by a single function call. Note that for some problems, notably poro-elasticity, this is not possible, then discretization must be carried out first.

Below, A is the global linear system, and b is the corresponding right hand side, and we obtain the pressure solution by solving the system.

In [8]:
dof_manager = pp.DofManager(gb)

assembler = pp.Assembler(gb, dof_manager)
assembler.discretize()

# Assemble the linear system, using the information stored in the GridBucket
A, b = assembler.assemble_matrix_rhs()

pressure = sps.linalg.spsolve(A, b)


The parameters assigned above will not yield a well-posed problem, thus the solve will likely produce a warning about the matrix being singular. This can be ignored. <br>

The ordering of the unknowns in the global linear system will vary depending on how the components in the GridBucket and the unknowns are traversed. Untangling the ordering is a two-stage process:
1. The system is ordered as a block system, with one block per combination of primary variable and grid or edge (between grids). This information is stored in the attribute `dof_manager.block_dof`.
2. For each block, the local degrees of freedom can be obtained from the attribute `dof_manager.full_dof`.

To get the block number of a specific primary variable, we need the identifier of the relevant component in the GridBucket (either the grid, or the edge between grids), and the variable name.

In [9]:
# Getting the grids is easy, there is one in each dimension
g_2d = gb.grids_of_dimension(2)[0]
g_1d = gb.grids_of_dimension(1)[0]

# Formally loop over the edges, there is a single one
for e, _ in gb.edges():
    continue

# Now, the block ordering is obtained, for the 2d grid as 
block_2d = dof_manager.block_dof[(g_2d, grid_variable)]

# full_dof contains the number of dofs per block. To get a global ordering, use
global_dof = np.r_[0, np.cumsum(dof_manager.full_dof)]

# Get 2d dofs
global_dof_2d = np.arange(global_dof[block_2d], global_dof[block_2d+1])
# Print the relevant part of the system matrix
print(A.toarray()[global_dof_2d, :][:, global_dof_2d])

[[ 50. -10.   0.   0.]
 [-10.  50.   0.   0.]
 [  0.   0.  50. -10.]
 [  0.   0. -10.  50.]]


# Example 2

The first example showed how to work with the assembler in reletively simple cases. In this second example, we aim to illustrate the full scope of the assembler, including:
* General assignment of variables on different grid components (fracture, matrix, etc.):
    * Different number of variables on each grid component
    * Different names for variables (a relevant case could be to use 'temperature' on one domain, 'enthalpy' on another, with an appropriate coupling)
* General coupling schemes between different grid components:
    * Multiple coupling variables
    * Couplings related to different variables and discretization schemes on the neighboring grids.
* Multiple discretization operators applied to the same term / equation on different grid components


The example that incorporates all these features are necessarily quite complex and heavy on notation. As such it should be considered as a reference for how to use the functionality, more than a simulation of any real physical system.

We define two primary variables on the nodes and three coupling variables. The resulting system will be somewhat arbitrary, in that it may not reflect any standard physics, but it should better illustrate what is needed for a multi-physics problem.

First we extend the data assignment method.

In [10]:
def assign_data_2(gb, keyword_param_storage, keyword_param_storage_2=None):
    # Method to assign data.
    for g, d in gb:
        # This keyword is used to define which set of default parameters to pick
        # Replace with 'transport' or 'mechanics' if needed
        default_parameter_type = 'flow'  

        # Assign a non-default permeability, for illustrative purposes
        if g.dim == 2:
            kxx = 10 * np.ones(g.num_cells)
        else:
            kxx = 0.1 * np.ones(g.num_cells)

        perm = pp.SecondOrderTensor(kxx)

        # Create a dictionary to override the default parameters
        specified_parameters = {'second_order_tensor': perm}

        #

        # Define the 
        pp.initialize_default_data(g, d, default_parameter_type, specified_parameters,
                                   keyword_param_storage)

        # Internally to the Parameter class, the parameters are stored as dictionaries.
        # To illustrate how to access specific sets of parameters, print the keywords
        # for one of the grids
        if g.dim == 2 and not keyword_param_storage_2:
            print('The assigned parameters for the 2d grid are')
            print(d[pp.PARAMETERS][keyword_param_storage].keys())
           
        # For one example below, we will need two different parameter sets.
        # Define a second set, with default values only.
        if keyword_param_storage_2:
            pp.initialize_default_data(g, d, default_parameter_type, keyword = keyword_param_storage_2)
        

    for e, d in gb.edges():
        # On edges in the GridBucket, there is currently no methods for default initialization.

        data = {"normal_diffusivity": 2e1}
        # Add parameters: We again use keywords to identify sets of parameters.
        if keyword_param_storage_2 is not None:
            # There are actually three parameters here ('two_parameter_sets' refers to the nodes)
            # since we plan on using in total three mortar variables in this case
            d[pp.PARAMETERS] = pp.Parameters(keywords=['flow_param_edge',
                                                       'second_flow_param_edge',
                                                       'third_flow_param_edge'],
                                             dictionaries=[data, data, data])
        else:
            d[pp.PARAMETERS] = pp.Parameters(keywords=['flow_param_edge'], dictionaries=[data])
    
    return gb

In [11]:
# Define a grid
gb, _ = pp.grid_buckets_2d.single_horizontal([4, 4], simplex=False)
parameter_keyword = 'flow_param'
parameter_keyword_2 = 'second_flow_param'
gb = assign_data_2(gb, parameter_keyword, parameter_keyword_2)

Primary variables must be defined on each component of the GridBucket.

On the first grid we use a cell centered method which has one primary variable "pressure". 
On the second grid, we use a mixed method with both pressure and fluxes combined into one primary variable.

The temperature is tagged with the same keyword on both grids.

In [12]:
# Variable keywords first grid
grid_1_pressure_variable = 'pressure'
grid_1_temperature_variable = 'temperature'
# Variable keywords second grid
grid_2_pressure_variable = 'flux_pressure'
grid_2_temperature_variable = 'temperature'

Next we assign a keyword to the coupling terms between the grid. We will have three coupling variables;
one for the fluid flux, and one for each of the diffusive terms in the temperature equation.

In [13]:
# Coupling variable for pressure
mortar_variable_pressure = 'mortar_flux_pressure'
# Coupling variable for advective temperature flux
mortar_variable_temperature_1 = 'mortar_flux_diffusion'
mortar_variable_temperature_2 = 'mortar_flux_diffusion_2'

We now give a keyword to the operators.

In [14]:
# Identifier of the discretization operator for pressure discretizaiton
operator_keyword_pressure = 'pressure_diffusion'

# identifier for the temperature discretizations.
# THIS IS WEIRD: The intention is to illustrate the use of two discretization operators for 
# a single variable. The natural option in this setting is advection-diffusion, but that
# requires either the existence of a Darcy flux, or tighter coupling with the pressure equation.
# Purely for illustrative purposes, we instead use a double diffusion model. There you go.
operator_keyword_temperature_1 = 'diffusion'
operator_keyword_temperature_2 = 'diffusion_2'

# Identifier of the discretization operator between grids
coupling_pressure_keyword = 'coupling_operator_pressure'

So far we have only defined the keywords needed for the discretizations to obtain the correct parameters
and couplings. Next, we create the discretization objects

In [15]:
# Pressure diffusion discretization
tpfa_flow = pp.Tpfa(parameter_keyword)
vem_flow = pp.MVEM(parameter_keyword)
# Temperature diffusion discretization
tpfa_temperature = pp.Tpfa(parameter_keyword_2)
mpfa_temperature = pp.Mpfa(parameter_keyword_2)

Discretization operators on the coupling conditions, chosen to illustrate the framework.
Note that in all cases, the coupling conditions need a separate keyword, which should 
correspond to an assigned set of data

In [16]:
# One term couples two pressure / flow variables
edge_discretization_flow = pp.RobinCoupling('flow_param_edge', tpfa_flow, vem_flow)

# The second coupling is of mpfa on one domain, and tpfa on the other, both for temperature
edge_discretization_temperature_diffusion_1 = pp.RobinCoupling('second_flow_param_edge',
                                                               mpfa_temperature, tpfa_temperature)

# The third coupling is of tpfa for flow with mpfa for temperature
edge_discretization_temperature_diffusion_2 = pp.RobinCoupling('third_flow_param_edge',
                                                               tpfa_flow, mpfa_temperature)

Loop over the nodes in the GridBucket, define primary variables and discretization schemes

In [17]:
for g, d in gb:
    # Assign primary variables on this grid. 
    if g.dim == 2:
        # Both pressure and temperature are represented as cell centered variables
        d[pp.PRIMARY_VARIABLES] = {grid_1_pressure_variable: {"cells": 1, "faces": 0},
                                   grid_1_temperature_variable: {"cells": 1}}

        # The structure of the discretization assignment is: For each variable, give a 
        # pair of operetor identifications (usually a string) and a discretizaiton method.
        # If a variable is identified with several discretizations, say, advection and diffusion,
        # several pairs can be assigned.
        
        # For pressure, use tpfa.
        # For temperature, use two discretizations, respectively tpfa and mpfa
        d[pp.DISCRETIZATION] = {grid_1_pressure_variable: {operator_keyword_pressure: tpfa_flow},
                                grid_1_temperature_variable: {operator_keyword_temperature_1: tpfa_temperature,
                                                              operator_keyword_temperature_2: mpfa_temperature}}
    else:  #g.dim == 1
        # Pressure is discretized with flux-pressure combination, temperature with cell centered variables
        d[pp.PRIMARY_VARIABLES] = {grid_2_pressure_variable: {"cells": 1, "faces": 1},
                                   grid_2_temperature_variable: {"cells": 1}}
        # For pressure, use vem.
        # For temperature, only discretize once, with tpfa
        d[pp.DISCRETIZATION] = {grid_2_pressure_variable: {operator_keyword_pressure: vem_flow},
                                grid_2_temperature_variable: {operator_keyword_temperature_1: tpfa_temperature}}

Loop over the edges in the GridBucket, define primary variables and discretizations.

Notice how coupling discretizations are assigned as a dictionary, one per coupling term on each edge. For each term, the coupling contains an inner dictionary, with the keys being the edge and the two neighboring grids. For the edge, the values are the name of the mortar variable, and the discretization object to be applied. For the grids, the values are the variable name on the grid, and the keyword identifying the discretization operator, as specified in the loop over nodes.

In [18]:
for e, d in gb.edges():
    # 
    g1, g2 = gb.nodes_of_edge(e)
    
    # The syntax used in the problem setup assumes that g1 has dimension 2
    if g1.dim < g2.dim:
        g2, g1 = g1, g2
                                
    # The mortar variable has one degree of freedom per cell in the mortar grid
    d[pp.PRIMARY_VARIABLES] = {mortar_variable_pressure: {"cells": 1},
                               mortar_variable_temperature_1: {"cells": 1},
                               mortar_variable_temperature_2: {"cells": 1},
                              }
    
    # Coupling discretizations
    d[pp.COUPLING_DISCRETIZATION] = {
        # The flow discretization couples tpfa on one domain with vem on the other
        'edge_discretization_flow': {
            g1: (grid_1_pressure_variable, operator_keyword_pressure),
            g2: (grid_2_pressure_variable, operator_keyword_pressure),
            e: (mortar_variable_pressure, edge_discretization_flow),
        },
        # The first temperature mortar couples one of the temperature discretizations on grid 1
        # with the single tempearture discretization on the second grid
        # As a side remark, the keys in the outer dictionary are never used, except from debugging,
        # but a dictionary seemed a more natural option than a list.
        'the_keywords_in_this_dictionary_can_have_any_value': {  
            g1: (grid_1_temperature_variable, operator_keyword_temperature_2),
            g2: (grid_2_temperature_variable, operator_keyword_temperature_1),
            e: (mortar_variable_temperature_1, edge_discretization_temperature_diffusion_1),
        },   
        # Finally, the third coupling
        'second_edge_discretization_temperature': {
            # grid_1_variable_1 gives pressure variable, then identify the discretization object
            g1: (grid_1_pressure_variable, operator_keyword_pressure),
            # grid_2_variable_2 gives temperature, then use the keyword that was used to identify mpfa
            # (and not the one for tpfa, would have been operator_keyword_temperature_1)
            g2: (grid_2_temperature_variable, operator_keyword_temperature_2),
            e: (mortar_variable_temperature_2, edge_discretization_temperature_diffusion_2),
        }       
        
    }
    d[pp.DISCRETIZATION_MATRICES] = {'flow_param_edge': {},
                                     'second_flow_param_edge': {},
                                     'third_flow_param_edge': {}
                                    }

We have now assigned all the data. The task of assembling the linear system is left to a dedicated object:

In [20]:
dof_manager = pp.DofManager(gb)
assembler = pp.Assembler(gb, dof_manager)

Discretization and assembly of the global linear system can again be carried out by separate function calls.

In [22]:
# Discretize, then Assemble the linear system, using the information stored in the GridBucket
assembler.discretize()
A, b = assembler.assemble_matrix_rhs()

# Pick out part of the discretization associated with the third mortar variable
g_2d = gb.grids_of_dimension(2)[0]

# Formally loop over the edges, there is a single one
for e, _ in gb.edges():
    continue

# Now, the block ordering is obtained, for the 2d grid as 
block_2d_pressure = dof_manager.block_dof[(g_2d, grid_1_pressure_variable)]
block_e_third_mortar = dof_manager.block_dof[(e, mortar_variable_temperature_2)]

# full_dof contains the number of dofs per block. To get a global ordering, use
global_dof = np.r_[0, np.cumsum(dof_manager.full_dof)]

# Get 2d dofs
global_dof_2d_pressure = np.arange(global_dof[block_2d_pressure], global_dof[block_2d_pressure+1])
global_dof_e_temperature = np.arange(global_dof[block_e_third_mortar], global_dof[block_e_third_mortar+1])
# Print the relevant part of the system matrix
print(A.toarray()[global_dof_2d_pressure, :][:, global_dof_e_temperature])

[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]
